In [1]:
"""IEEE Test Case."""
from pathlib import Path
import pandas as pd
import pomato

# Init POMATO with the options file and the dataset
wdir = Path.cwd() # wdir should refer to this examples folder
mato = pomato.POMATO(wdir=wdir, options_file="profiles/ieee118.json")
mato.load_data('data_input/pglib_opf_case118_ieee.m')

INFO - Creating Folder Structure
INFO - Initializing DataObject
INFO - Initializing the GridTopology....
INFO - Initializing MarketModel...
INFO - Initializing the FBMCModule....
INFO - Initializing the GridTopology....
INFO - POMATO initialized!
INFO - Loading data from matpower .m case-file
INFO - Slackbus read as 69
INFO - Validating Input Data...
WARNING - Data validation completed with warnings. See the .data_validation_report.
WARNING - Some data was initialized empty. See model_validation_report.
WARNING - Some data missing or contained NaNs. See model_validation_report.
INFO - Calculating grid parameters!
INFO - The network consists of 1 components. Setting slack for each.
INFO - Calculating PTDF and PSDF matrices!
INFO - Checking GridTopology...
INFO - Radial nodes: Contingency of 7 lines is set to false
INFO - Radial lines: Contingency of 2 lines is set to false
INFO - Total number of lines and contingencies: 186, 177
INFO - Calculating LODF matrix!
INFO - Calculating conting

In [2]:
# %% Access the data from the main pomato instance in the data object.
nodes = mato.data.nodes
lines = mato.grid.lines
demand = mato.data.demand_el
zones = mato.data.zones
plants = mato.data.plants


In [3]:
# Return the results, in this case two: 
## %% Run uniform pricing
mato.options["type"] = "dispatch"
mato.create_grid_representation()
mato.run_market_model()

# Obtain the market result by name and its instance
dispatch_result_name = mato.market_model.result_folders[0].name
dispatch_result = mato.data.results[dispatch_result_name]

# Check Overloaded Lines for N-0 and N-1 contingency cases.
df1, df2 = dispatch_result.overloaded_lines_n_0()
df3, df4 = dispatch_result.overloaded_lines_n_1()
print("Number of overloaded lines (Dispatch): ", len(df1))
print("Number of overloaded lines N-1 (Dispatch): ", len(df3))


INFO - No grid representation needed for dispatch model
INFO - Start-Time: 15:20:43
Starting Julia Process... INFO - Loading Gurobi...
INFO - Initialized
Starting Julia Process... -INFO - reading from file c:\users\riw\documents\repositories\pomato\pomato/daemon_market_model.json
INFO - Done with Initialization. Starting daemon process market_model!
INFO - Process ready!
Processing... \INFO - Loading Gurobi...
INFO - Starting with market_model
INFO - Run market model
INFO - No logfile Found, logging only to console.
INFO - Read Model Data..
INFO - Reading Model Data from: c:\Users\riw\Documents\repositories\pomato\examples/data_temp/julia_files/data/
INFO - Data Prepared
INFO - Creating Results Folder
INFO - Initializing Market Model for timestep t0001...
INFO - Academic license - for non-commercial use only
INFO - Adding Variables and Expressions..
INFO - Add Base Model
INFO - Adding Objective Function...
INFO - Solving...
INFO - Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)


In [4]:
# Check for overloaded lines in the market and redispatch results
# %% Run nodal-prcing market clearing
mato.options["type"] = "nodal"
mato.create_grid_representation()
mato.run_market_model()

# Obtain the market result by name and its instance
nodal_result_name = mato.market_model.result_folders[0].name
nodal_result = mato.data.results[nodal_result_name]

# Check overloaded lines for N-0 and N-1 contingency cases.
df1, df2 = nodal_result.overloaded_lines_n_0()
df3, df4 = nodal_result.overloaded_lines_n_1()

print("Number of overloaded lines (nodal): ", len(df1))
print("Number of overloaded lines N-1 (nodal): ", len(df3))


INFO - Start-Time: 15:22:19
Starting Julia Process... INFO - Process ready!
Processing... INFO - Loading Gurobi...
INFO - Starting with market_model
INFO - Run market model
INFO - No logfile Found, logging only to console.
INFO - Read Model Data..
INFO - Reading Model Data from: c:\Users\riw\Documents\repositories\pomato\examples/data_temp/julia_files/data/
INFO - Data Prepared
INFO - Creating Results Folder
INFO - Initializing Market Model for timestep t0001...
INFO - Academic license - for non-commercial use only
INFO - Adding Variables and Expressions..
INFO - Add Base Model
INFO - Adding power flow constraints using the angle formulation.
INFO - Adding Objective Function...
INFO - Solving...
INFO - Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
INFO - Optimize a model with 630 rows, 864 columns and 2186 nonzeros
INFO - Model fingerprint: 0x4af95c32
INFO - Coefficient statistics:
INFO - Matrix range     [1e+00, 5e+04]
INFO - Objective range  [1e+00, 1e+02]
INFO - Bounds rang

In [5]:
mato.options["type"] = "cbco_nodal"
mato.options["grid"]["redundancy_removal_option"] = "clarkson_base"

# Requires to presolve the network with the RedundancyRemoval Algorithm
mato.create_grid_representation()

# Update the model data
mato.update_market_model_data()
mato.run_market_model()

# Check for overloaded lines (Should be none for N-0 and N-1)
scopf_result_name = mato.market_model.result_folders[0].name
scopf_result = mato.data.results[scopf_result_name]

df1, df2 = scopf_result.overloaded_lines_n_0()
df3, df4 = scopf_result.overloaded_lines_n_1()

print("Number of overloaded lines (SCOPF): ", len(df1))
print("Number of overloaded lines N-1 (SCOPF): ", len(df3))

# Create geoplot for scopf result, including price layer and saving it as .html
mato.visualization.create_geo_plot(scopf_result, show_prices=True, show_plot=False)



04.06 - 15:23:34 on Thread: 13
INFO - ################    80%- Size of I 32 - Found Redundant 82 - Remaining 29 - 04.06 - 15:23:34 on Thread: 1
INFO - ################    80%- Size of I 32 - Found Redundant 82 - Remaining 29 - 04.06 - 15:23:34 on Thread: 10
INFO - Nonredundant indices 34 from process: 12
INFO - Nonredundant indices 42 from process: 7
INFO - ################    80%- Size of I 24 - Found Redundant 106 - Remaining 33 - 04.06 - 15:23:34 on Thread: 14
INFO - Nonredundant indices 30 from process: 2
INFO - Nonredundant indices 52 from process: 11
INFO - Nonredundant indices 44 from process: 9
INFO - Nonredundant indices 46 from process: 3
INFO - Nonredundant indices 39 from process: 6
INFO - Nonredundant indices 48 from process: 5
INFO - Nonredundant indices 51 from process: 13
INFO - Nonredundant indices 38 from process: 1
INFO - Nonredundant indices 41 from process: 10
INFO - Nonredundant indices 61 from process: 4
INFO - Nonredundant indices 39 from process: 14
INFO - Leng

In [6]:
mato._join_julia_instances()


INFO - EXIT
INFO - EXIT
